In [35]:
'''
This project aims to differentiate between 2 lists of companies and extract the names of companies which are similar between them.
The catch is that some of the names might be misspelled. 
Hence to determine whether the companies are considered similar we will:
1) First extract the names which are exactly the same between the 2 lists
2) Secondly, extract names which are substrings of a name in the other list
3) Thirdly, calculate the Levenshtein distance function from the textdistance module and include the names if the similarity score >0.9 (arbitrary cut off)
4) Lastly, check if there are names which are substrings of another found name within the same list. The reason this step is required is because the time required for Step 3 is large, hence once a match has been found, the loop moves on to the next name

References:
https://yassineelkhal.medium.com/the-complete-guide-to-string-similarity-algorithms-1290ad07c6b7
https://www.geeksforgeeks.org/python-test-if-string-is-subset-of-another/
'''

In [ ]:
#import the necessary libraries
import textdistance as td
import pandas as pd

In [36]:
#Then we read in the data into a pandas dataframe
df = pd.read_csv('raw.csv')
df.head()

,Unnamed: 0,JM,Unnamed: 2,CSL
0,NaN,NaN,NaN,NaN
1,1.0,100 Super Minimart,1.0,101 EXPRESSMART PTE LTD
2,2.0,13 Mart Pte Ltd,2.0,13MART PTE LTD
3,3.0,1588 LE Pte Ltd (New Econ),3.0,1588 LE PTE LTD
4,4.0,199 Budget Shop,4.0,178 WANG TRADING


The columns we require are in the JM and CSL columns

In [37]:
#Cleaning the data by removing null values in the column and changing the strings to lowercase
jm = df.JM
jm.dropna(inplace = True)
jm = jm.str.lower()
jm.shape

(1815,)

In [38]:
jm.head()

1            100 super minimart
2               13 mart pte ltd
3    1588 le pte ltd (new econ)
4               199 budget shop
5             21 beyond pte ltd
Name: JM, dtype: object

In [39]:
#Cleaning the data by removing null values in the column and changing the strings to lowercase
csl = df.CSL
csl.dropna(inplace = True)
csl = csl.str.lower()
csl.shape

(930,)

In [40]:
csl.head()

1    101 expressmart pte ltd
2             13mart pte ltd
3            1588 le pte ltd
4          178 wang trading 
5            199 budget shop
Name: CSL, dtype: object

In [41]:
#First we extract the names which are similar between both lists
j = set(jm)
c = set(csl)
s = j&c


In [42]:
#If one is a substring of another, we will include it 
sj = j-s
sc = c-s

d = dict()

for a in sc:
    for b in sj:
        if a in d.keys():
            break
        else:
            if a in b or b in a: #We consider them to be similar if one string is a subset of the other
                d[a] = b

In [43]:
#Then we can use the textdistance module to determine if 2 names are similar to each other

sjd = sj-set(d.values())
scd = sc-set(d.keys())

for a in scd:
    for b in sjd:
        if a in d.keys():
            break
        else:
            if td.levenshtein.normalized_similarity(a,b) > 0.9: #We consider them to be similar if the similarity score is >0.9 (arbitrary cut-off)
                d[a] = b



In [46]:
#Combine the names which have been found

df1 = pd.DataFrame.from_dict( {'CSL':d.keys(),'JM':d.values()})

df2 = pd.DataFrame.from_dict( {'CSL':[x for x in s],'JM':[x for x in s]})

fdf = pd.concat([df2,df1]).reset_index(drop=True)


In [47]:
#We can then find if there are names which are substrings of another found name within the same list

remaining_jm = sjd-set(d.values())
remaining_csl = scd-set(d.keys())

rj = set()
rc = set()

for a in remaining_jm:
    if fdf.JM.str.contains(a,regex = False).sum() > 0:
        rj.add(a)

for a in remaining_csl:
    if fdf.CSL.str.contains(a,regex = False).sum() > 0:
        rc.add(a)

df_rj = pd.DataFrame.from_dict( {'JM':[x for x in rj]})
df_rc = pd.DataFrame.from_dict( {'CSL':[x for x in rc]})

rdf = pd.concat([df_rj,df_rc], axis = 1).reset_index(drop = True)


In [48]:
#Add these into the names which have been found
fdf = pd.concat([fdf,rdf]).reset_index(drop=True)

In [50]:
#find the mismatches

j_mm = j - set(fdf.JM)
c_mm = c - set(fdf.CSL)

df_jmm = pd.DataFrame.from_dict( {'JM_Not_Found':[x for x in j_mm]})
df_cmm = pd.DataFrame.from_dict( {'CSL_Not_Found':[x for x in c_mm]})

df_mm = pd.concat([df_cmm,df_jmm], axis = 1).reset_index(drop = True)

df_mm

,CSL_Not_Found,JM_Not_Found
0,seng yong huat prov & fruits shop,sri vinayaga trading
1,seven star minimart,jade minimart
2,l3 home-matters pte ltd,krish impex pte ltd
3,aik seng (s)trading,zafrah mini mart
4,zhen tat pte. ltd.,jsa trading pte ltd
...,...,...
1310,NaN,express avenue
1311,NaN,smart alliance
1312,NaN,eco family mart
1313,NaN,leo's minimart


In [52]:
#Combine the matched and mismatched and output as an excel file

fdf = pd.concat([fdf,df_mm], axis = 1).reset_index(drop = True)

fdf.to_excel("clean.xlsx")  
